<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 815ms/step - loss: 0.6588 - accuracy: 0.6144 - val_loss: 0.6165 - val_accuracy: 0.7854
Epoch 2/10
26/26 [==============================] - 20s 765ms/step - loss: 0.5882 - accuracy: 0.7056 - val_loss: 0.5579 - val_accuracy: 0.6927
Epoch 3/10
26/26 [==============================] - 20s 780ms/step - loss: 0.5515 - accuracy: 0.7324 - val_loss: 0.6347 - val_accuracy: 0.5805
Epoch 4/10
26/26 [==============================] - 20s 769ms/step - loss: 0.4715 - accuracy: 0.8078 - val_loss: 0.5416 - val_accuracy: 0.7073
Epoch 5/10
26/26 [==============================] - 20s 773ms/step - loss: 0.4093 - accuracy: 0.8321 - val_loss: 0.3788 - val_accuracy: 0.8488
Epoch 6/10
26/26 [==============================] - 20s 769ms/step - loss: 0.3798 - accuracy: 0.8467 - val_loss: 0.3464 - val_accuracy: 0.8878
Epoch 7/10
26/26 [==============================] - 20s 773ms/step - loss: 0.3258 - accuracy: 0.8796 - val_loss: 0.2620 - val_accuracy: 0.9366

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.65.h5
26/26 - 21s - loss: 0.6638 - accuracy: 0.6204 - val_loss: 0.6468 - val_accuracy: 0.6049
Epoch 2/5

Epoch 00002: saving model to weights.02-0.69.h5
26/26 - 20s - loss: 0.6262 - accuracy: 0.6679 - val_loss: 0.6945 - val_accuracy: 0.5171
Epoch 3/5

Epoch 00003: saving model to weights.03-0.55.h5
26/26 - 20s - loss: 0.5835 - accuracy: 0.6861 - val_loss: 0.5469 - val_accuracy: 0.7902
Epoch 4/5

Epoch 00004: saving model to weights.04-0.54.h5
26/26 - 20s - loss: 0.5239 - accuracy: 0.7470 - val_loss: 0.5355 - val_accuracy: 0.7366
Epoch 5/5

Epoch 00005: saving model to weights.05-0.78.h5
26/26 - 20s - loss: 0.4844 - accuracy: 0.7908 - val_loss: 0.7814 - val_accuracy: 0.5171


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6645 - accuracy: 0.6071 - val_loss: 0.6517 - val_accuracy: 0.5512


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 21s - loss: 0.6688 - accuracy: 0.5803 - val_loss: 0.6586 - val_accuracy: 0.5659
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 21s - loss: 0.6210 - accuracy: 0.6740 - val_loss: 0.5896 - val_accuracy: 0.7902


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6843 - accuracy: 0.5280 - val_loss: 0.6852 - val_accuracy: 0.4390
Epoch 2/50
26/26 - 20s - loss: 0.6675 - accuracy: 0.5523 - val_loss: 0.6682 - val_accuracy: 0.5268
Epoch 3/50
26/26 - 20s - loss: 0.6447 - accuracy: 0.6618 - val_loss: 0.6421 - val_accuracy: 0.6098
Epoch 4/50
26/26 - 20s - loss: 0.6087 - accuracy: 0.7457 - val_loss: 0.5893 - val_accuracy: 0.8390
Epoch 5/50
26/26 - 20s - loss: 0.5487 - accuracy: 0.8090 - val_loss: 0.5359 - val_accuracy: 0.7951
Epoch 6/50
26/26 - 21s - loss: 0.4838 - accuracy: 0.8127 - val_loss: 0.4449 - val_accuracy: 0.8732
Epoch 7/50
26/26 - 20s - loss: 0.4105 - accuracy: 0.8394 - val_loss: 0.4730 - val_accuracy: 0.7512
Epoch 8/50
26/26 - 21s - loss: 0.3413 - accuracy: 0.8808 - val_loss: 0.2732 - val_accuracy: 0.9463
Epoch 9/50
26/26 - 20s - loss: 0.2686 - accuracy: 0.9124 - val_loss: 0.2192 - val_accuracy: 0.9415
Epoch 10/50
26/26 - 20s - loss: 0.2207 - accuracy: 0.9282 - val_loss: 0.1851 - val_accuracy: 0.9659
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 825ms/step - loss: 0.6670 - accuracy: 0.6131 - val_loss: 0.6623 - val_accuracy: 0.5415
Epoch 2/5
26/26 [==============================] - 21s 799ms/step - loss: 0.6216 - accuracy: 0.6740 - val_loss: 0.6612 - val_accuracy: 0.5366
Epoch 3/5
26/26 [==============================] - 21s 819ms/step - loss: 0.5901 - accuracy: 0.6776 - val_loss: 0.6413 - val_accuracy: 0.5659
Epoch 4/5
26/26 [==============================] - 21s 800ms/step - loss: 0.5486 - accuracy: 0.7324 - val_loss: 0.6360 - val_accuracy: 0.5854
Epoch 5/5
26/26 [==============================] - 20s 788ms/step - loss: 0.5161 - accuracy: 0.7664 - val_loss: 0.5170 - val_accuracy: 0.7073


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.613139,0.666843,0.541463,0.662274
1,1,0.673966,0.621882,0.536585,0.661184
2,2,0.677616,0.590176,0.565854,0.641273
3,3,0.732360,0.548162,0.585366,0.636019
4,4,0.766423,0.517224,0.707317,0.516965


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 31s 1s/step - loss: 0.6809 - accuracy: 0.5523 - val_loss: 0.6952 - val_accuracy: 0.4390

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 788ms/step - loss: 0.6494 - accuracy: 0.6363 - val_loss: 0.6623 - val_accuracy: 0.5171

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 785ms/step - loss: 0.6368 - accuracy: 0.6618 - val_loss: 0.6416 - val_accuracy: 0.6780

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 21s 808ms/step - loss: 0.6282 - accuracy: 0.7092 - val_loss: 0.6441 - val_accuracy: 0.6049

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 21s 819ms/step - loss: 0.6245

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 24s 942ms/step - loss: 0.6674 - accuracy: 0.5925 - val_loss: 0.6863 - val_accuracy: 0.4634
Epoch 2/50
26/26 [==============================] - 21s 822ms/step - loss: 0.6227 - accuracy: 0.6861 - val_loss: 0.6171 - val_accuracy: 0.6634
Epoch 3/50
26/26 [==============================] - 21s 811ms/step - loss: 0.5532 - accuracy: 0.7409 - val_loss: 0.5274 - val_accuracy: 0.7512
Epoch 4/50
26/26 [==============================] - 21s 815ms/step - loss: 0.5019 - accuracy: 0.7640 - val_loss: 0.4677 - val_accuracy: 0.7805
Epoch 5/50
25/26 [===========================>..] - ETA: 0s - loss: 0.4333 - accuracy: 0.8062
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 22s 846ms/step - loss: 0.4341 - accuracy: 0.8078 - val_loss: 0.5201 - val_accuracy: 0.7317
Epoch 6/50
26/26 [==============================] - 21s 812ms/step - loss: 0.3841 - accuracy: 0.8552 - val_loss: 0.3744 - 

In [ ]:
%tensorboard --logdir log_dir